In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn jdata numpy

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import jdata as jd
from mediapipe.framework.formats import landmark_pb2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
atlas10_5=jd.load('atlas_19-5_10-5.json')

In [3]:
def affinemap(pfrom, pto):
    bsubmat=np.eye(3)
    ptnum=len(pfrom)
    amat=np.zeros((ptnum*3,9))
    for i in range(ptnum):
        amat[((i+1)*3-3):((i+1)*3),:]=np.kron(bsubmat,pfrom[i,:])
    amat=np.hstack((amat,np.tile(bsubmat,(ptnum,1))))
    
    bvec=np.reshape(pto,(ptnum*3,1))
    x=np.linalg.lstsq(amat, bvec, rcond=None)[0]
    A=np.reshape(x[0:9],(3,3))
    b=x[9:12]
    return [A, b]

def reg1020(Amat, bvec, pts):
    newpt = np.matmul(Amat, (np.array(pts)).T)
    newpt = newpt + np.tile(bvec, (1, len(pts)))
    newpt = newpt.T
    return newpt

def landmark2numpy(landmarks):
    pts = []
    for p in landmarks.landmark:
        pts.append([p.x, p.y, p.z])
    return np.array(pts)

def numpy2landmark(pts):
    landmarks = landmark_pb2.NormalizedLandmarkList(landmark=[])
    for p in pts:
         lm =  landmark_pb2.NormalizedLandmark(x=p[0], y=p[1], z=p[2])
         landmarks.landmark.append(lm)
    return landmarks

In [6]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, smooth_landmarks=True) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False    

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if(results.face_landmarks is not None):
            pts=results.face_landmarks.landmark
            landmark_face_subset = landmark_pb2.NormalizedLandmarkList(
                landmark=[
                    pts[168],
                    pts[10],
                    {'x':2*pts[234].x-pts[227].x, 'y':2*pts[234].y-pts[227].y, 'z':2*pts[234].z-pts[227].z},
                    {'x':2*pts[454].x-pts[447].x, 'y':2*pts[454].y-pts[447].y, 'z':2*pts[454].z-pts[447].z}
                ])
            #print(landmark2numpy(landmark_face_subset))
            #print(np.array([atlas10_5['nz'], atlas10_5['sm'][0], atlas10_5['rpa'], atlas10_5['lpa']]))
            Amat, bvec = affinemap(np.array([atlas10_5['nz'], atlas10_5['sm'][1], atlas10_5['rpa'], atlas10_5['lpa']]),
                                   landmark2numpy(landmark_face_subset))

            aal = reg1020(Amat, bvec, atlas10_5['aal'])
            aar = reg1020(Amat, bvec, atlas10_5['aar'])
            sm = reg1020(Amat, bvec, atlas10_5['sm'])
            cm = reg1020(Amat, bvec, atlas10_5['cm'])

            
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )   
        if(results.face_landmarks is not None):
            mp_drawing.draw_landmarks(image, landmark_face_subset, None, 
                                 mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=3),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=3)
                                 )   
            mp_drawing.draw_landmarks(image, numpy2landmark(np.vstack((aal,aar,sm,cm))), None, 
                                 mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=3),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=3)
                                 )   
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            jd.save(facelandmarks, 'facelandmarks.json')
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'facelandmarks' is not defined

In [106]:
atlas10_5


{'nz': [87.4123, 188.512, 93.7087],
 'iz': [87.436, 5.9213, 116.0523],
 'lpa': [21.1737, 93.9561, 84.9446],
 'rpa': [159.644, 89.8472, 86.3139],
 'cz': [84.62214999524781, 104.6686683166328, 214.239394704542],
 'cm': [[11.44149454085169, 97.26684264500733, 115.7191341721597],
  [10.72187124264941, 98.98416586638984, 133.0371959194771],
  [11.77338948661696, 100.6429981215048, 150.3121432934689],
  [16.03082231783018, 102.1526959564027, 167.0671185701407],
  [25.05911913554962, 103.3273494470869, 181.8928169132195],
  [37.45911478975918, 104.1446284774095, 194.1231603532999],
  [51.63623899403721, 104.6989300246092, 204.2233312349896],
  [67.47523150794045, 104.9227934655142, 211.468106897651],
  [84.62214999524781, 104.6686683166328, 214.239394704542],
  [101.8624728192473, 103.9163277399205, 211.9503995704584],
  [118.3645719039702, 102.8805872176999, 206.535254269509],
  [133.0232638524031, 101.5258506314215, 197.2843837122807],
  [145.670112440762, 99.97886018146193, 185.43979931873

In [102]:
mp_face_mesh = mp.solutions.face_mesh  # initialize the face mesh model

# camera stream:
cap = cv2.VideoCapture(0)  # chose camera index (try 1, 2, 3)
with mp_face_mesh.FaceMesh(
        max_num_faces=1,  # number of faces to track in each frame
        refine_landmarks=True,  # includes iris landmarks in the face mesh model
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:  # no frame input
            print("Ignoring empty camera frame.")
            continue
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # frame to RGB for the face-mesh model
        results = face_mesh.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # frame back to BGR for OpenCV
        
        inion = results.multi_face_landmarks[0].landmark[8]
        leftear = results.multi_face_landmarks[0].landmark[93]
        rightear = results.multi_face_landmarks[0].landmark[323]
        fp1 = results.multi_face_landmarks[0].landmark[10]
        
        facekeypt = landmark_pb2.NormalizedLandmarkList(
          landmark=[inion, leftear, rightear]
        )

        mp_drawing.draw_landmarks(image, facekeypt, None, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=2)
                                 )   
        cv2.imshow('output window', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()

TypeError: 'NoneType' object is not subscriptable